In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import rasterio

# Function to preprocess DEM data
def preprocess_dem(dem):
    # Normalize the DEM data (values between 0 and 1)
    normalized_dem = dem / np.max(dem)
    # Scale the DEM data to 0-255 for visualization
    scaled_dem = (normalized_dem * 255).astype(np.uint8)
    return scaled_dem

# Function to identify slopes using edge Canny detection
def identify_slopes(dem):
    # Perform edge detection using Canny on the preprocessed DEM
    edges = cv2.Canny(dem, 50, 150)

    # Calculate gradient using Sobel operator
    gradient_x = cv2.Sobel(dem, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(dem, cv2.CV_64F, 0, 1, ksize=3)

    # Calculate magnitude and angle of the gradient
    magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    angle = np.arctan2(gradient_y, gradient_x) * (180 / np.pi)

    # Set threshold for slopes greater than 10 degrees
    slope_threshold = 10
    steep_slope_mask = np.zeros_like(dem)
    steep_slope_mask[np.abs(angle) > slope_threshold] = 255  # Using grayscale value for visualization

    return steep_slope_mask  # Return the mask for slopes greater than 10 degrees

# Load DEM datasets using rasterio
dem_dir_1 = '/kaggle/input/nacdem'
dem_dir_2 = '/kaggle/input/tmcdtm'

dem_files_1 = [os.path.join(dem_dir_1, f) for f in os.listdir(dem_dir_1) if f.endswith('.tif')]
dem_files_2 = [os.path.join(dem_dir_2, f) for f in os.listdir(dem_dir_2) if f.endswith('.tif')]

preprocessed_dem1 = []
preprocessed_dem2 = []

# Load and preprocess DEM datasets
for dem_file in dem_files_1:
    with rasterio.open(dem_file) as dem:
        dem_array = dem.read(1)
        preprocessed_dem1.append(preprocess_dem(dem_array))

for dem_file in dem_files_2:
    with rasterio.open(dem_file) as dem:
        dem_array = dem.read(1)
        preprocessed_dem2.append(preprocess_dem(dem_array))

# Identify slopes for both DEM datasets
slope_masks_1 = [identify_slopes(dem) for dem in preprocessed_dem1]
slope_masks_2 = [identify_slopes(dem) for dem in preprocessed_dem2]

# Display original images and sample slope images for both datasets with user-defined figure sizes
num_samples = 1
figsize_1 = (15, 6)  # Change figure size for DEM dataset 1
figsize_2 = (40, 20)  # Change figure size for DEM dataset 2

plt.figure(figsize=figsize_1)

# For DEM dataset 1
for i in range(num_samples):
    with rasterio.open(dem_files_1[i]) as dem:
        image = dem.read(1)
        plt.subplot(3, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f'DEM 1 Original {i+1}')
        plt.axis('off')
    
    plt.subplot(3, num_samples, num_samples + i + 1)
    plt.imshow(slope_masks_1[i], cmap='gray')  # Grayscale colormap for better edge detection
    plt.title(f'DEM 1 Sample Slope {i+1}')
    plt.axis('off')
    # Save slope mask for DEM dataset 1
    plt.imsave(f'dem1_slope_{i+1}.png', slope_masks_1[i], cmap='gray')

plt.tight_layout()
plt.show()

plt.figure(figsize=figsize_2)

# For DEM dataset 2
for i in range(num_samples):
    with rasterio.open(dem_files_2[i]) as dem:
        image = dem.read(1)
        plt.subplot(3, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f'DEM 2 Original {i+1}')
        plt.axis('off')

    plt.subplot(3, num_samples, num_samples + i + 1)
    plt.imshow(slope_masks_2[i], cmap='gray')  # Grayscale colormap for better edge detection
    plt.title(f'DEM 2 Sample Slope {i+1}')
    plt.axis('off')
    # Save slope mask for DEM dataset 2
    plt.imsave(f'dem2_slope_{i+1}.png', slope_masks_2[i], cmap='gray')

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/nacdem'

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import rasterio

# Function to detect shadows using intensity comparison
def detect_shadows(image):
    # Calculate a threshold based on intensity (adjust the value as needed)
    threshold_value = 150  # Adjust this threshold value for shadow detection
    _, thresh = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresh

# Load ORTH datasets using rasterio
orth_dir_1 = '/kaggle/input/orthonac123/ORTHONAC0.5-1/1'
orth_dir_2 = '/kaggle/input/tmcorth/TMCORTHO5-1/1'

orth_files_1 = [os.path.join(orth_dir_1, f) for f in os.listdir(orth_dir_1) if f.endswith('.tif')]
orth_files_2 = [os.path.join(orth_dir_2, f) for f in os.listdir(orth_dir_2) if f.endswith('.tif')]

# Process images and detect shadows
shadow_masks_1 = []
shadow_masks_2 = []

for orth_file in orth_files_1:
    with rasterio.open(orth_file) as orth:
        image = orth.read(1)  # Read a single-band grayscale image
        # Detect shadows in the image
        shadow_mask = detect_shadows(image)
        shadow_masks_1.append(shadow_mask)

for orth_file in orth_files_2:
    with rasterio.open(orth_file) as orth:
        image = orth.read(1)  # Read a single-band grayscale image
        # Detect shadows in the image
        shadow_mask = detect_shadows(image)
        shadow_masks_2.append(shadow_mask)

# Display sample images with shadow masks
num_samples = 1
figsize_1 = (7, 6)  # Change figure size for ORTH dataset 1
figsize_2 = (40, 20)  # Change figure size for ORTH dataset 2

plt.figure(figsize=figsize_1)

# For ORTH dataset 1
for i in range(num_samples):
    with rasterio.open(orth_files_1[i]) as orth:
        image = orth.read(1)  # Read a single-band grayscale image
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title('Original Image 1')
        plt.axis('off')

        plt.subplot(2, num_samples, num_samples + i + 1)
        plt.imshow(shadow_masks_1[i], cmap='gray')
        plt.title('Shadow Mask 1')
        plt.axis('off')
        
        # Save shadow mask
        plt.imsave(f'shadow_mask_1_{i}.png', shadow_masks_1[i], cmap='gray')  # Save the shadow mask

plt.tight_layout()
plt.show()

plt.figure(figsize=figsize_2)

# For ORTH dataset 2
for i in range(num_samples):
    with rasterio.open(orth_files_2[i]) as orth:
        image = orth.read(1)  # Read a single-band grayscale image
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title('Original Image 2')
        plt.axis('off')

        plt.subplot(2, num_samples, num_samples + i + 1)
        plt.imshow(shadow_masks_2[i], cmap='gray')
        plt.title('Shadow Mask 2')
        plt.axis('off')
        
        # Save shadow mask
        plt.imsave(f'shadow_mask_2_{i}.png', shadow_masks_2[i], cmap='gray')  # Save the shadow mask

plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import Image
import cv2
import albumentations as A  # our data augmentation library

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# remove arnings (optional)
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm # progress bar
from torchvision.utils import draw_bounding_boxes

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
# our dataset is in cocoformat, we will need pypcoco tools
!pip install pycocotools
from pycocotools.coco import COCO

In [ ]:
# Now, we will define our transforms
from albumentations.pytorch import ToTensorV2

In [ ]:
def get_transforms(train=False):
    if train:
        transform = A.Compose([
            A.Resize(600, 600), # our input size can be 600px
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.1),
            A.ColorJitter(p=0.1),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    else:
        transform = A.Compose([
            A.Resize(600, 600), # our input size can be 600px
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    return transform

In [ ]:
class AquariumDetection(datasets.VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        # the 3 transform parameters are reuqired for datasets.VisionDataset
        super().__init__(root, transforms, transform, target_transform)
        self.split = split #train, valid, test
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json")) # annotatiosn stored here
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]
    
    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))
    
    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))
        
        boxes = [t['bbox'] + [t['category_id']] for t in target] # required annotation format for albumentations
        if self.transforms is not None:
            transformed = self.transforms(image=image, bboxes=boxes)
        
        image = transformed['image']
        boxes = transformed['bboxes']
        
        new_boxes = [] # convert from xywh to xyxy
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.tensor(new_boxes, dtype=torch.float32)
        
        targ = {} # here is our transformed target
        targ['boxes'] = boxes
        targ['labels'] = torch.tensor([t['category_id'] for t in target], dtype=torch.int64)
        targ['image_id'] = torch.tensor([t['image_id'] for t in target])
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) # we have a different area
        targ['iscrowd'] = torch.tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        return image.div(255), targ # scale images
    def __len__(self):
        return len(self.ids)

In [ ]:
dataset_path = "/kaggle/input/yolov7"


In [ ]:
#load classes
coco = COCO(os.path.join(dataset_path, "train", "_annotations.coco.json"))
categories = coco.cats
n_classes = len(categories.keys())
categories

In [ ]:
classes = [i[1]['name'] for i in categories.items()]
classes

In [ ]:
train_dataset = AquariumDetection(root=dataset_path, transforms=get_transforms(True))

In [ ]:
num_samples_to_display = 5  # Set the number of samples to display

fig, axes = plt.subplots(1, num_samples_to_display, figsize=(20, 10))

for i in range(num_samples_to_display):
    sample = train_dataset[i]  # Access each sample from the dataset
    img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)  # Convert image to tensor

    # Plot each image with bounding boxes
    axes[i].imshow(draw_bounding_boxes(
        img_int, sample[1]['boxes'], [classes[i] for i in sample[1]['labels']], width=4
    ).permute(1, 2, 0))
    axes[i].axis('off')  # Turn off axis labels
    
plt.tight_layout()
plt.show()


In [ ]:
len(train_dataset)

In [ ]:
# lets load the faster rcnn model
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features # we need to change the head
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=collate_fn)


In [ ]:
images,targets = next(iter(train_loader))
images = list(image for image in images)
targets = [{k:v for k, v in t.items()} for t in targets]
output = model(images, targets) # just make sure this runs without error

In [ ]:
device = torch.device("cuda")
model = model.to(device)


In [ ]:
# Now, and optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)
# lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[16, 22], gamma=0.1) # lr scheduler

In [ ]:
import sys

In [ ]:
def train_one_epoch(model, optimizer, loader, device, epoch):
    model.to(device)
    model.train()
    
#     lr_scheduler = None
#     if epoch == 0:
#         warmup_factor = 1.0 / 1000 # do lr warmup
#         warmup_iters = min(1000, len(loader) - 1)
        
#         lr_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor = warmup_factor, total_iters=warmup_iters)
    
    all_losses = []
    all_losses_dict = []
    
    for images, targets in tqdm(loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets) # the model computes the loss automatically if we pass in targets
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
        
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping trainig") # train if loss becomes infinity
            print(loss_dict)
            sys.exit(1)
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
#         if lr_scheduler is not None:
#             lr_scheduler.step() # 
        
    all_losses_dict = pd.DataFrame(all_losses_dict) # for printing
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    # Assuming lr_scheduler is active
    # lr_scheduler.step() 


In [ ]:
test_dataset = AquariumDetection(root=dataset_path, split="test", transforms=get_transforms(False))
model.eval()
torch.cuda.empty_cache()

num_images_to_display = 5
count_displayed = 0

for i in range(len(test_dataset)):
    img, _ = test_dataset[i]
    with torch.no_grad():
        prediction = model([img.to(device)])
        pred = prediction[0]

    if count_displayed < num_images_to_display and len(pred['labels']) > 0:
        boxes_to_plot = pred['boxes'][pred['scores'] > 0.3]
        labels_to_plot = [classes[i] for i in pred['labels'][pred['scores'] > 0.2].tolist()]

        if len(labels_to_plot) != len(boxes_to_plot):
            print(f"Number of boxes and labels mismatch for image {i}")
            labels_to_plot = labels_to_plot[:len(boxes_to_plot)]

        img_int = torch.tensor(img * 255, dtype=torch.uint8)
        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        ax.imshow(draw_bounding_boxes(img_int, boxes_to_plot, labels_to_plot, width=4).permute(1, 2, 0))
        ax.set_title('Image with Bounding Boxes')
        ax.axis('off')
        plt.show()

        crater_masks = []
        boulder_masks = []
        for bbox, label in zip(boxes_to_plot, labels_to_plot):
            xmin, ymin, xmax, ymax = map(int, bbox.tolist())
            mask = np.zeros((img_int.shape[1], img_int.shape[2]), dtype=np.uint8)
            if label == 'crater':  # Detect and create a mask for craters
                mask[ymin:ymax, xmin:xmax] = 1
                crater_masks.append(mask)
            elif label == 'boulder':  # Detect and create a mask for boulders
                mask[ymin:ymax, xmin:xmax] = 1
                boulder_masks.append(mask)

        num_masks_to_display = len(crater_masks) + len(boulder_masks)
        fig, axes = plt.subplots(1, num_masks_to_display + 1, figsize=(20, 8))
        axes[0].imshow(img_int.permute(1, 2, 0))
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        for j, mask in enumerate(crater_masks + boulder_masks):
            axes[j + 1].imshow(mask, cmap='gray')
            axes[j + 1].set_title(f'Mask {j+1}')
            axes[j + 1].axis('off')

        plt.tight_layout()
        plt.show()

        count_displayed += 1

    if count_displayed >= num_images_to_display:
        break


In [ ]:
num_samples = 2  # Number of samples to consider

# Assuming masks are available
if slope_masks_1 and shadow_masks_1:
    resized_slope_masks = [cv2.resize(mask, (img_int.shape[2], img_int.shape[1])) for mask in slope_masks_1]
    resized_shadow_masks = [cv2.resize(mask, (img_int.shape[2], img_int.shape[1])) for mask in shadow_masks_1]

if slope_masks_2 and shadow_masks_2:
    resized_slope_masks_2 = [cv2.resize(mask, (img_int.shape[2], img_int.shape[1])) for mask in slope_masks_2]
    resized_shadow_masks_2 = [cv2.resize(mask, (img_int.shape[2], img_int.shape[1])) for mask in shadow_masks_2]
    
    # Combine slope and shadow masks for Dataset 2
    combined_masks_2 = []
    for slope_mask, shadow_mask in zip(resized_slope_masks_2, resized_shadow_masks_2):
        # Combine masks by logical OR operation
        combined_mask = np.maximum(slope_mask, shadow_mask)
        combined_masks_2.append(combined_mask)

    # Display the combined hazard map for Dataset 2
    plt.figure(figsize=(15, 6))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(combined_masks_2[i], cmap='RdYlGn')  
        plt.title('Combined Hazard Map (Dataset 2)')
        plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("Slope or Shadow masks for Dataset 2 are not available.")


In [ ]:
from tkinter import *
from tkinter import filedialog
import tkinter as tk
from PIL import Image, ImageTk
def quit_program():
    root.destroy()

def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif")])
    if file_path:
        print(f"Selected Image: {file_path}")
        display_image(file_path)

def display_image(file_path):
    # Create a new frame
    new_frame = tk.Toplevel(root)
    new_frame.title("Selected Image")

    # Display the image in the new frame
    image = Image.open(file_path)
    image.thumbnail((300, 300))  # Resize the image for display
    photo = ImageTk.PhotoImage(image)
    image_label = tk.Label(new_frame, image=photo)
    txt_=tk.Label(new_frame,text=file_path)
    txt_.pack()
    image_label.image = photo
    image_label.pack()

def create_new_frame():
    new_frame = tk.Toplevel(root)
    new_frame.title("Hazard Map Generation Precision Lunar Lander")
    def quit_page():
        new_frame.destroy()
    image = Image.open("D:\SIH\dem1.jpg")  
    image.thumbnail((400, 400))
    photo = ImageTk.PhotoImage(image)
    image_label = Label(new_frame, image=photo)
    image_label.photo = photo
    image_label.grid(row=0, column=0, pady=(10, 20), padx=20)
    text_label = tk.Label(new_frame, text="D:\SIH\dem1.jpg")   
    text_label.grid(row=1, column=0, pady=(0, 10), padx=20)

    image1 = Image.open("D:\SIH\dem2.jpg")   
    image1.thumbnail((400, 400))
    photo1 = ImageTk.PhotoImage(image1)
    image_label1 = Label(new_frame, image=photo1)
    image_label1.photo1 = photo1
    image_label1.grid(row=0, column=1, pady=(10, 20), padx=20)
    text_label1 = tk.Label(new_frame, text="D:\SIH\dem2.jpg")   
    text_label1.grid(row=1, column=1, pady=(0, 10), padx=20)

    image2 = Image.open("D:\SIH\Shadow1.jpg")   
    image2.thumbnail((400, 400))
    photo2 = ImageTk.PhotoImage(image2)
    image_label2 = Label(new_frame, image=photo2)
    image_label2.photo2 = photo2
    image_label2.grid(row=0, column=2, pady=(10, 20), padx=20)
    text_label2 = tk.Label(new_frame, text="D:\SIH\Shadow1.jpg")   
    text_label2.grid(row=1, column=2, pady=(0, 10), padx=20)

    image3 = Image.open("D:\SIH\shadow2.jpg")   
    image3.thumbnail((400, 400))
    photo3 = ImageTk.PhotoImage(image3)
    image_label3 = Label(new_frame, image=photo3)
    image_label3.photo3 = photo3
    image_label3.grid(row=2, column=1, pady=(10, 20), padx=20)
    text_label3 = tk.Label(new_frame, text="D:\SIH\shadow2.jpg")   
    text_label3.grid(row=3, column=1, pady=(0, 10), padx=20)

    quit_button = Button(new_frame, text="EXIT",padx=50,pady=10, command=quit_program)
    quit_button.grid(row=2, column=2, pady=(10, 10), padx=20)

    back_button = Button(new_frame, text="BACK",padx=50,pady=10, command=quit_page)
    back_button.grid(row=2, column=0, pady=(10, 10), padx=20)

def move_gif(frame_num=0):
    try:
        # Open the GIF and get the next frame
        gif_img.seek(frame_num)
        gif_frame = ImageTk.PhotoImage(gif_img)

        # Update the background image
        background_label.config(image=gif_frame)
        background_label.image = gif_frame

        # Schedule the next frame update
        root.after(50, move_gif, frame_num + 1)
    except EOFError:
        # Restart the GIF loop when it reaches the end
        move_gif()


root = tk.Tk()
root.title('Orion Space Systems')

gif_img = Image.open("D:\SIH\luna1.gif")  # Replace with your actual GIF file path
background_label = tk.Label(root)
background_label.place(relwidth=1, relheight=1)
move_gif()

# Create buttons
image_button = tk.Button(root, text="UPLOAD IMAGE", padx=100, pady=40,bg="#565650",fg="white", command=open_image)
image_button.grid(row=1, column=1, pady=45, padx=40)

tot=tk.Label(root,text="ORION SPACE SYSTEMS\n\nGENERATION OF HAZARD MAP FOR LANDER",font=("Gotham",11),bg="black",fg="white",padx=50,pady=30)
tot.grid(row=0, column=10, columnspan=7, pady=40,padx=700)

toti=tk.Label(root,text="UPLOAD AN IMAGE\n USING THE BUTTON BELOW\n TO CONDUCT A COMPREHENSIVE ANALYSIS\n OF THE LUNAR TMC IMAGE,\n TO IDENTIFY POTENTIAL THREATS AND CREATING\n AN HAZARD MAP TO STRENGTHEN\n SAFETY PROTOCOLS FOR THE LANDER.",font=("din condensed",9),bg="#565650",fg="white",padx=10,pady=30)
toti.grid(row=0, column=1, columnspan=7, pady=90,padx=40)

button = tk.Button(root, text="GENERATE", padx=110, pady=40,bg="#565650",fg="white", command=create_new_frame)
button.grid(row=1, column=10, columnspan=7, pady=12,padx=700)  # Use grid method

totn=tk.Label(root,text="INITIATE THE DATA PROCESSING BY\n CLICKING THE 'GENERATE' BUTTON\n ABOVE TO ANALYZE THE IMPORTED TMC DATA.\n THIS CRITICAL STEP WILL ENABLE\n THE CREATION OF A VITAL HAZARD MAP ESSENTIAL\n FOR THE SAFETY PROTOCOL OF THE LANDER.",font=("din condensed",9),bg="#565650",fg="white",padx=10,pady=30)
totn.grid(row=2, column=10, columnspan=7, pady=1,padx=700)

# Run the main loop
root.mainloop()